from Daniel Brown


I recently combined the MTA Bus Route Segment Speeds dataset with the MTA General Transit Feed Specification (GTFS) Static Data to create detailed visualizations of bus speeds across route segments in NYC.

The tricky part was getting the data into the right format. Geospatial data is tricky to work with, and aligning stops with shapes took some careful preparation.

In [ ]:
import pandas as pd
import json
from math import radians, sin, cos, sqrt, atan2

# --- 1. Load GTFS shapes and trips ---
shapes = pd.read_csv("/Users/danielbrown/Desktop/gtfs_bx-3/shapes.txt")
shapes = shapes.sort_values(["shape_id", "shape_pt_sequence"])

trips = pd.read_csv("/Users/danielbrown/Desktop/gtfs_bx-3/trips.txt")

# --- 2. Load speeds ---
speeds = pd.read_csv(
    "/Users/danielbrown/Desktop/MTA_Bus_Route_Segment_Speeds__Beginning_2025_20250920.csv",
    parse_dates=["Timestamp"]
)

# --- 2b. Aggregate speeds by Next Timepoint Stop Name and Direction ---
agg_speeds = (
    speeds.groupby(["Next Timepoint Stop Name", "Direction"], as_index=False)
    .agg(
        avg_speed=("Average Road Speed", "mean"),
        total_trips=("Bus Trip Count", "sum"),
        start_lat=("Timepoint Stop Latitude", "first"),
        start_lon=("Timepoint Stop Longitude", "first"),
        end_lat=("Next Timepoint Stop Latitude", "first"),
        end_lon=("Next Timepoint Stop Longitude", "first"),
        route_id=("Route ID", "first")
    )
)

# --- 2c. Map directions and add segment numbers ---
dir_map = {'W': 0, 'E': 1}  # adjust if needed
agg_speeds['direction_id'] = agg_speeds['Direction'].map(dir_map)

# Number segments within each route + direction
agg_speeds = agg_speeds.sort_values(["route_id", "direction_id"])
agg_speeds["segment_number"] = (
    agg_speeds.groupby(["route_id", "direction_id"]).cumcount() + 1
)

# --- 3. Helper functions ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi, dlambda = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1) * cos(phi2) * sin(dlambda/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

def closest_index(lat, lon, coords):
    return min(range(len(coords)), key=lambda i: haversine(lat, lon, coords[i][1], coords[i][0]))

# --- 4. Map route+direction to shape_id ---
direction_map = trips.groupby(['route_id', 'direction_id'])['shape_id'].first().to_dict()

# --- 5. Build GeoJSON ---
features = []

for _, row in agg_speeds.iterrows():
    route_dir_key = (row["route_id"], row["direction_id"])
    if route_dir_key not in direction_map:
        continue
    shape_id = direction_map[route_dir_key]
    route_shape = shapes[shapes["shape_id"] == shape_id]
    shape_coords = list(zip(route_shape["shape_pt_lon"], route_shape["shape_pt_lat"]))

    # Find indices along shape for stop and next stop
    i1 = closest_index(row["start_lat"], row["start_lon"], shape_coords)
    i2 = closest_index(row["end_lat"], row["end_lon"], shape_coords)
    if i1 > i2:
        i1, i2 = i2, i1

    # Fallback if points are the same
    if i1 == i2:
        segment_coords = [(row["start_lon"], row["start_lat"]), (row["end_lon"], row["end_lat"])]
    else:
        segment_coords = shape_coords[i1:i2+1]

    features.append({
        "type": "Feature",
        "geometry": {"type": "LineString", "coordinates": segment_coords},
        "properties": {
            "route_id": row["route_id"],
            "direction": row["Direction"],
            "segment_number": int(row["segment_number"]),
            "speed": row["avg_speed"],
            "trips": row["total_trips"]
        }
    })

geojson = {"type": "FeatureCollection", "features": features}

with open("/Users/danielbrown/Desktop/bx12_speeds_shapes_directions_agg_1.geojson", "w") as f:
    json.dump(geojson, f)

print(f"GeoJSON created with {len(features)} features")